# Práctica 1 : Titanic

La finalidad de la práctica es entrenar una red neuronal para predecir la supervivencia o muerte (0 o 1) de un pasajero del Titanic a partir de otras características, como su edad, sexo, clase...  
El objetivo es introducirnos en las redes neuronales y empezar a comprender mejor como funcionan.

Eric Salazar Moreira
12-03-2024

## Importaciones y carga del dataset


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

df = pd.read_csv('train.csv')

## Exploración inicial

In [ ]:
## Una vista a la estructura general del dataframe
print(df.head())

In [ ]:
## Saber el número total de filas y columnas
num_filas, num_columnas = df.shape
print("Número total de filas:", num_filas)
print("Número total de columnas:", num_columnas)

In [ ]:
## Saber el tipo de datos de cada columna
tipos_de_datos = df.dtypes
print(tipos_de_datos)

In [ ]:
## Saber si contamos con valores nulos en las columnas
valores_nulos = df.isnull().sum()
print(valores_nulos)

In [ ]:
## Un resumen estadístico de las variables numéricas
summary = df.describe()
print(summary)

In [ ]:
## Ver si en las columnas numéricas hay ceros
columnas_numericas = df.select_dtypes(include=['int', 'float'])
registros_con_0 = (columnas_numericas == 0).sum()
print(registros_con_0)

In [ ]:
## Ver los valores únicos en las columnas categóricas
columnas_caracteres = df.select_dtypes(include=['object'])
valores_unicos_caracteres = columnas_caracteres.nunique()
print(valores_unicos_caracteres)

En la exploración inicial podemos ver que la distribución de los datos es buena, que con el campo Sex puede hacerse one-hot-encoding y que los registros nulos en el campo Age parecen preocupantes, ya que de primeras parece una variable importante a tener en cuenta para predecir la supervivencia. También hay muchos campos que pueden sobrar, como ticket, cabin, embarked... Puesto que no son relevantes para que una persona sobreviva.

## Eliminación de columnas y renombres

In [ ]:
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
df = df.rename(columns={"Survived": "Supervivencia","Pclass": "Clase","Sex": "Sexo","Age": "Edad","SibSp": "Hermanos","Parch": "Padres_hijos","Fare": "Tarifa", })

df.head(2)

He eliminado las columnas que a mi parecer no afectaban a la supervivencia de los pasajeros y hacían que se tuviese que trabajar con más datos que no ayudaban al futuro modelo. También he renombrado las columnas para tenerlas en español y que me sea más sencillo trabajar con ellas.

## Conversión de datos
Con tal de solo tener datos numéricos y sabiendo que la columna "Sexo" es binaria, convertiré los datos de male a 0 y de female a 1 en vez de hacer one-hot-encoding

In [ ]:
df['Sexo'] = df['Sexo'].replace({'female': 1, 'male': 0})
df['Sexo'] = df['Sexo'].astype(int)

## Imputación de datos
En la columna "Edad" contamos con 177 valores nulos. A continuación voy a ver qué hacer con este problema.

In [ ]:
df['Edad'].mean()
correlacion1 = df['Edad'].corr(df['Supervivencia'])
print("Correlación entre edad y supervivencia:", correlacion1)

In [ ]:
df_sin_nulos = df.dropna(subset=['Edad'])
valores_nulos = df_sin_nulos.isnull().sum()
print(valores_nulos)
df_sin_nulos['Edad'].mean()
num_filas, num_columnas = df_sin_nulos.shape
print("Número total de filas:", num_filas)
print("Número total de columnas:", num_columnas)
correlacion2 = df_sin_nulos['Edad'].corr(df_sin_nulos['Supervivencia'])
print("Correlación entre edad y supervivencia:", correlacion2)
# Crear el histograma de edades
plt.figure(figsize=(8, 6))
plt.hist(df_sin_nulos['Edad'], bins=20, color='skyblue', edgecolor='black')

# Ajustar etiquetas y título
plt.title('Distribución de Edades')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

# Mostrar el histograma
plt.show()

In [ ]:
media_edad = df['Edad'].mean()
df_media = df.copy()
df_media['Edad'].fillna(media_edad, inplace=True)
valores_nulos = df_media.isnull().sum()
print(valores_nulos)
num_filas, num_columnas = df_media.shape
print("Número total de filas:", num_filas)
print("Número total de columnas:", num_columnas)
correlacion3 = df_sin_nulos['Edad'].corr(df_sin_nulos['Supervivencia'])
print("Correlación entre edad y supervivencia:", correlacion3)

# Crear el histograma de edades
plt.figure(figsize=(8, 6))
plt.hist(df_media['Edad'], bins=20, color='skyblue', edgecolor='black')

# Ajustar etiquetas y título
plt.title('Distribución de Edades')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

# Mostrar el histograma
plt.show()


In [ ]:
mediana_edad = df['Edad'].median()
df_mediana = df.copy()
df_mediana['Edad'].fillna(media_edad, inplace=True)
valores_nulos = df_mediana.isnull().sum()
print(valores_nulos)
num_filas, num_columnas = df_mediana.shape
print("Número total de filas:", num_filas)
print("Número total de columnas:", num_columnas)
correlacion3 = df_sin_nulos['Edad'].corr(df_sin_nulos['Supervivencia'])
print("Correlación entre edad y supervivencia:", correlacion3)

# Crear el histograma de edades
plt.figure(figsize=(8, 6))
plt.hist(df_mediana['Edad'], bins=20, color='skyblue', edgecolor='black')

# Ajustar etiquetas y título
plt.title('Distribución de Edades')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

# Mostrar el histograma
plt.show()

Basándome en las gráficas y viendo como se distribuyen las edades al imputarlas tanto con media como con mediana, he decidido que lo más correcto es eliminar los registros sin edad del dataframe, esto se debe a que imputando con media o mediana se pierde mucho la distribución normal de las edades reales y se crea un pico demasiado denso, los registros sin edad tampoco comportan una parte muy importante del dataframe y borrarlos no afecta al nivel de correlación, como se puede comprobar. Si hubiesen datos a partir de los cuales poder predecir la edad, sería una buena práctica hacer una imputacion calculada, pero en este caso no es posible.

In [ ]:
df = df_sin_nulos.copy()
df.shape

## Correlación en los datos

In [ ]:
df_numeric = df.select_dtypes(include=['int', 'float'])
plt.figure(figsize=(10, 5))
corr = df_numeric.corr()
sns.heatmap(corr, cmap="BrBG", annot=True)
plt.show()

No hay una relación muy amplia entre la supervivencia de los pasajeros y sus características, lo más destacable es que el sexo sí que afecta a la supervivencia de los pasajeros, igual que la tarifa que pagaron por el viaje, el resto de campos no afectan a la supervivencia.


## Separación del conjunto de datos

In [ ]:
df.head(1)
# Dividir los datos en características (X) y la variable objetivo (y)
X = df.drop('Supervivencia', axis=1)  # características
y = df['Supervivencia']  # variable objetivo

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba (70% entreno, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


He dividido los datos en X e Y y en conjuntos para entrenamiento y prueba, un 70% para entreno y un 30% para testing. Donde: 
1. X_train: características de entrenamiento
2. X_test: características de prueba
3.  y_train: etiquetas de entrenamiento
4. y_test: etiquetas de prueba

## Creación del modelo

In [ ]:
# Definir la arquitectura de la red neuronal
model = Sequential()

# Añadir la primera capa oculta
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))


# Añadir la capa de salida
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


He creado el modelo con los requisitos definidos en la práctica, activación relu, salida sigmoid puesto que es un resultado binario y optimizador adam. Solo una capa oculta puesto que añadiendo más de una bajaba el rendimiento, ya que eran excesivas para los pocos datos con los que la red está tratando

## Entrenamiento del modelo y visualización de resultados

In [ ]:
# Entrenar el modelo
# Obtener la historia de entrenamiento
history = model.fit(X_train, y_train, epochs=4, batch_size=32, validation_data=(X_test, y_test))

# Obtener la pérdida y la precisión durante el entrenamiento
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

# Plotear la pérdida
plt.plot(epochs, train_loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plotear la precisión
plt.plot(epochs, train_accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print("Pérdida en el conjunto de prueba:", loss)
print("Precisión en el conjunto de prueba:", accuracy)

## Conclusiones

Viendo los plots y los resultados del modelo, puedo llegar a varias conclusiones:

1. **Rendimiento general**: El modelo muestra una precisión del 76.74% en el conjunto de prueba, lo que indica una capacidad razonable para capturar patrones en los datos.

2. **Estabilidad durante el entrenamiento**: A lo largo de las cuatro épocas, tanto la pérdida como la precisión en el conjunto de validación muestran una tendencia estable, sugiriendo que el modelo no está sobreajustando o subajustando en exceso.

3. **Posible sobreajuste**: Aunque la brecha entre la precisión en el conjunto de entrenamiento y el conjunto de validación sugiere cierto sobreajuste, este efecto parece ser moderado y manejable.

4. **Ajuste adicional**: Dado que la precisión en el conjunto de validación no ha alcanzado un pico y la pérdida no ha convergido completamente, podría ser beneficioso explorar un entrenamiento adicional para mejorar aún más el rendimiento.

En resumen, el modelo funciona bien y cumple con su cometido, pero existe margen para modificar parámetros y mejorar su rendimiento.